28-Input and Output 之九
====
**Binary Format Description**

# numpy.lib.format

Binary serialization

# NPY format

一种简单的格式，用于将numpy数组保存到磁盘，并提供有关它们的完整信息。

`.npy`格式用于在 NumPy中将单个NumPy数组以标准二进制文件格式持久化保存到磁盘。该格式存储了即使在具有不同体系结构的另一台机器上也能正确重建阵列所需的所有形状和dtype信息。格式设计尽可能简单，同时实现其有限的目标。

`.npz`格式是用于在Numpy中将多个NumPy数组以标准格式持久化到磁盘。一个`.npz`文件是包含多个`.npy`文件的压缩文件，每个`.npy`为一个数组。

A simple format for saving numpy arrays to disk with the full information about them.

The .npy format is the standard binary file format in NumPy for persisting a single arbitrary NumPy array on disk. The format stores all of the shape and dtype information necessary to reconstruct the array correctly even on another machine with a different architecture. The format is designed to be as simple as possible while achieving its limited goals.

The .npz format is the standard format for persisting multiple NumPy arrays on disk. A .npz file is a zip file containing multiple .npy files, one for each array.

## 功能 Capabilities

- 可以表示所有NumPy数组，包括嵌套记录数组和对象数组。
- 表示其本机二进制形式的数据。
- 直接支持Fortran连续数组。
- 存储所有必要的信息以重建阵列，包括在不同架构的机器上的形状和dtype。支持little-endian和big-endian数组，带有little-endian数字的文件将在读取文件的任何机器上产生little-endian数组。这些类型根据其实际尺寸进行描述。例如，如果具有64位C“long int”的机器写出具有“long int”的数组，则具有32位C“long int”的读取机将产生具有64位整数的数组。
- 逆向工程是直截了当的。数据集通常比创建它们的程序寿命更长。一个称职的开发人员应该能够用他们喜欢的编程语言创建一个解决方案，以便.npy在没有太多文档的情况下阅读他给出的大多数文件
- 允许数据的内存映射。请参阅open_memmep。
- 可以从类似文件的流对象而不是实际文件中读取。
- 存储对象数组，即包含任意Python对象元素的数组。具有对象数组的文件不是mmapable，但可以读取和写入磁盘。


- Can represent all NumPy arrays including nested record arrays and object arrays.
- Represents the data in its native binary form.
- Supports Fortran-contiguous arrays directly.
- Stores all of the necessary information to reconstruct the array including shape and dtype on a machine of a different architecture. Both little-endian and big-endian arrays are supported, and a file with little-endian numbers will yield a little-endian array on any machine reading the file. The types are described in terms of their actual sizes. For example, if a machine with a 64-bit C “long int” writes out an array with “long ints”, a reading machine with 32-bit C “long ints” will yield an array with 64-bit integers.
- Is straightforward to reverse engineer. Datasets often live longer than the programs that created them. A competent developer should be able to create a solution in their preferred programming language to read most .npy files that he has been given without much documentation.
- Allows memory-mapping of the data. See open_memmep.
- Can be read from a filelike stream object instead of an actual file.
- Stores object arrays, i.e. arrays containing elements that are arbitrary Python objects. Files with object arrays are not to be mmapable, but can be read and written to disk.

## Limitations 限制

- `numpy.ndarray`的任意子类未完全保留。将接受子类进行写入，但只会写出数组数据。读取文件后将创建一个常规的numpy.ndarray对象。

- Arbitrary subclasses of numpy.ndarray are not completely preserved. Subclasses will be accepted for writing, but only the array data will be written out. A regular numpy.ndarray object will be created upon reading the file.

**Warning**

由于结构化dtypes的解释有限，具有空名称字段的dtypes将使用'f0'，'f1'等替换名称。此类数组不会完全准确地往返于格式。数据完好无损; 只有字段名称会有所不同。我们正在努力解决这个问题。此修复程序不需要更改文件格式。具有这种结构的数组仍然可以保存和恢复，并且可以使用该`loadedarray.view(correct_dtype)`方法恢复正确的dtype 。


Due to limitations in the interpretation of structured dtypes, dtypes with fields with empty names will have the names replaced by ‘f0’, ‘f1’, etc. Such arrays will not round-trip through the format entirely accurately. The data is intact; only the field names will differ. We are working on a fix for this. This fix will not require a change in the file format. The arrays with such structures can still be saved and restored, and the correct dtype may be restored by using the `loadedarray.view(correct_dtype)` method.

## File extensions 文件扩展名

建议使用`.npy`和`.npz`作为文件的扩展名这样的格式保存文件。这绝不是必要条件; 应用程序可能希望使用这些文件格式，但使用特定于应用程序的扩展。然而，在没有明显替代方案的情况下，我们建议使用.npy和.npz。

We recommend using the .npy and .npz extensions for files saved in this format. This is by no means a requirement; applications may wish to use these file formats but use an extension specific to the application. In the absence of an obvious alternative, however, we suggest using .npy and .npz.

## Version numbering 版本编号

这些格式的版本编号与NumPy版本编号无关。如果格式升级，numpy.io中的代码仍然可以读取和写入1.0版文件。

The version numbering of these formats is independent of NumPy version numbering. If the format is upgraded, the code in numpy.io will still be able to read and write Version 1.0 files.

## Format Version 1.0

前6个字节是一个魔术字符串：完全正确\x93NUMPY。

下一个1字节是无符号字节：文件格式的主要版本号，例如\x01。

下一个1字节是无符号字节：文件格式的次要版本号，例如\x00。注意：文件格式的版本与numpy软件包的版本无关。

接下来的2个字节构成一个little-endian unsigned short int：头数据HEADER_LEN的长度。

下一个HEADER_LEN字节构成描述数组格式的头数据。它是一个ASCII字符串，包含字典的Python文字表达式。它由换行符（\ n \n）终止并用空格（\x20）填充，以使总数 可被64整除，以便进行对齐。len(magic string) + 2 + len(length) + HEADER_LEN

该词典包含三个键：

The first 6 bytes are a magic string: exactly \x93NUMPY.

The next 1 byte is an unsigned byte: the major version number of the file format, e.g. \x01.

The next 1 byte is an unsigned byte: the minor version number of the file format, e.g. \x00. Note: the version of the file format is not tied to the version of the numpy package.

The next 2 bytes form a little-endian unsigned short int: the length of the header data HEADER_LEN.

The next HEADER_LEN bytes form the header data describing the array’s format. It is an ASCII string which contains a Python literal expression of a dictionary. It is terminated by a newline (\n) and padded with spaces (\x20) to make the total of len(magic string) + 2 + len(length) + HEADER_LEN be evenly divisible by 64 for alignment purposes.

The dictionary contains three keys:

**“descr” : dtype.descr**

>可以作为参数传递给numpy.dtype 构造函数以创建数组的dtype的对象。

>An object that can be passed as an argument to the numpy.dtype constructor to create the array’s dtype.

**“fortran_order” : bool**

>数组数据是否为Fortran连续。由于Fortran连续数组是非C连续性的常见形式，因此我们允许将它们直接写入磁盘以提高效率。

>Whether the array data is Fortran-contiguous or not. Since Fortran-contiguous arrays are a common form of non-C-contiguity, we allow them to be written directly to disk for efficiency.

**“shape” : tuple of int**

>数组的形状。

>The shape of the array.

为了重复性和可读性，字典键按字母顺序排序。这只是为了方便。作家应尽可能实现这一点。读者不得依赖于此。

标题后面是数组数据。如果dtype包含Python对象（即`dtype.hasobject is True`），那么数据就是数组的Python pickle。否则，数据是阵列的连续（C- or Fortran-, 取决于 `fortran_order`）字节。用户可以通过将元素的数量与`dtype.itemsize`给出的shape相乘计算出字节数（注意  `shape=()`意味着有 1 个元素）

乘以形状给出的元素数量来计算出字节数（注意这意味着有1个元素）。dtype.hasobject is Truefortran_ordershape=()dtype.itemsize

For repeatability and readability, the dictionary keys are sorted in alphabetic order. This is for convenience only. A writer SHOULD implement this if possible. A reader MUST NOT depend on this.

Following the header comes the array data. If the dtype contains Python objects (i.e. `dtype.hasobject is True`), then the data is a Python pickle of the array. Otherwise the data is the contiguous (either C- or Fortran-, depending on `fortran_order`) bytes of the array. Consumers can figure out the number of bytes by multiplying the number of elements given by the shape (noting that `shape=()` means there is 1 element) by `dtype.itemsize`.

## Format Version 2.0

1.0版格式只允许数组头的总大小为65535字节。具有大量列的结构化数组可能会超出此范围。2.0版格式将标头大小扩展为4 GiB。 如果数据需要，`numpy.save`将自动保存为2.0格式，否则它将始终使用更兼容的1.0格式。

因此，对标题的第四个元素的描述变为：“接下来的4个字节形成一个小端无符号整数：标题数据HEADER_LEN的长度。”

The version 1.0 format only allowed the array header to have a total size of 65535 bytes. This can be exceeded by structured arrays with a large number of columns. The version 2.0 format extends the header size to 4 GiB. numpy.save will automatically save in 2.0 format if the data requires it, else it will always use the more compatible 1.0 format.

The description of the fourth element of the header therefore has become: “The next 4 bytes form a little-endian unsigned int: the length of the header data HEADER_LEN.”

**注意：**

`.npy`格式，包括创造它的动机和可选方案的比较，在[“npy-format”NEP](http://www.numpy.org/neps/nep-0001-npy-format.html)中做了描述，但细节已经演变随着时间的推移而进化，并且这个文件也更新。

The .npy format, including motivation for creating it and a comparison of alternatives, is described in the “npy-format” NEP, however details have evolved with time and this document is more current.